In [79]:
import json
from functools import cmp_to_key
import functools
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re
import gzip
import zlib
def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(idx)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others
g=len
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y = re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            a = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(a)):
                c=c.replace(a[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
    
document3 = """import json as a,sys,re
from functools import cmp_to_key
g=len
s=sorted
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y=re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            d = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(d)):
                c=c.replace(d[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
n,o=sys.argv[1:3]
k,v=a.load(open(n))
w,x=t(k)
y,z=t(v)
c=s(zip(w,x,k),key=cmp_to_key(p))
b=s(zip(y,z,v),key=cmp_to_key(p))
a.dump(dict([(c[i][2],b[i][2]) for i in range(g(c))]),open(o,'w'))"""

def extract_remove(lst):
    remove, extract = [], []
    for i in range(len(lst)):
        integers = re.findall('([0-9]+)',lst[i])
        if len(integers) != 0:
            integer_lst = list(re.finditer('([0-9]+)',lst[i]))
            cpy = lst[i][:]
            for j in range(len(integer_lst)):
                cpy = cpy.replace(integer_lst[j].group(0),'',1)
            remove.append(cpy)
        else:
            remove.append(lst[i])
        extract.append(integers)
    return extract, remove

def shift_count(ori,new):
    count = 0
    for k,v in ori.items():
        if new[k] != v:
            count += 1
    print('shift count:',count)
    return count

def bit_size(lst):
    maximum = 0
    for item in lst:
        all_int = re.findall('\d+',item)
        if not all_int:
            continue
        maximum = max(maximum, (len(all_int[0]) if len(all_int) == 1 else max([*map(len,all_int)])))        
    return maximum

def zfill_method(origin_mapping,maximum):
    # ori=[k:('','',...),v:('','',...)] 
    ori=[*zip(*origin_mapping.items())]

    # ori_zfill =[0:['','',...],1:['','',...]]
    ori_zfill=[*map(lambda x:[*map(lambda y:re.sub(r'(\d+)',lambda z:z.group(1).zfill(maximum),y),x)],ori)]

    # ori_and_zfill = [[(k1,kf1),(k2,kf2),...],[(v1,vf1),(v2,vf2),...]]
    ori_and_zfill=[*map(lambda x,y:[*zip(x,y)],ori,ori_zfill)]

    # sort_by_zfill = [[(k1,kf1),(k2,kf2),...],[(v1,vf1),(v2,vf2),...]]
    sort_by_zfill=[*map(lambda x:sorted(x,key=lambda y:y[1]),ori_and_zfill)]

    # transpose=[0:[(k1,k2,...),(kf1,kf2,...)],1:[(v1,v2,...),(vf1,vf2,...)]] 
    transpose = [*map(lambda x: [*zip(*x)],sort_by_zfill)]
    new_table=dict(zip(transpose[0][0],transpose[1][0]))
    return shift_count(origin_mapping,new_table), generate_idx_list([*zip(*ori_zfill)])

def generate_idx_list(pack):
    pack.sort(key=lambda x: x[0]) 
    k,v = zip(*pack)        
    mm = sorted(zip(v,[i for i in range(len(k))]),key=lambda x: x[0])
    v, idx = zip(*mm) 
    idx = list(idx)
    print(len(str(idx)))
    mark = consecu_int_run_length(idx)
    print('consecu idx list char count: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
    return mark,sporadic_seg

def check_type(origin_mapping):
    # check largest bit size
    maximum = max(bit_size(origin_mapping.keys()),bit_size(origin_mapping.values()))
    print('bit_size:',maximum)
    same, prefix, infix, postfix, others = classify(origin_mapping)
    
    #CASE0
    flag = 0
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    for i in range(len(keys)):
        if values[i] != origin_mapping[keys[i]]:
            flag = 1
            break
    
    print('default type')
    idx=generate_shift_table(origin_mapping,[keys,values])
    res, readble = generate_run_length_table(idx)
    two_level_run_length(res)
    if flag == 0:
        print('pure type')
        return 0
    '''
    # zfill_method
    if not zfill_method(origin_mapping,maximum)[0]:
        print('zfill type')
        return 1
    '''
    if natural_key_method(origin_mapping)==0:
        return 1
    
    return 2
def A(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1

def natural_key(x):
    return [int(c)if c.isdigit()else c for c in re.split('\d+',x)]

def natural_key_method(origin_mapping):
    ori = [list(origin_mapping.keys()),list(origin_mapping.values())]
    new = dict(zip(*map(lambda x:sorted(x,key=natural_key),ori)))
    print('using natural key')
    idx=generate_shift_table(origin_mapping,[list(new.keys()),list(new.values())])
    res, readble = generate_run_length_table(idx)
    print()
    two_level_run_length(res)
    return shift_count(origin_mapping,new)

#B:dimensions(string,i)
#C:order(string)
#D:dictionary_order(rhs,lhs)
#E:iter_bit(dim,rhs,lhs)
#F:dimension_based(rhs,lhs)
#S:string
#R:rhs
#L:lhs
def dimensions(string,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order(string):
    it=re.findall('\$O([0-9]+)',string)
    return int(it[0]) if len(it) else 0
def dictionary_order(R,L):
    return 1 if R>L else 0 if R==L else -1
def iter_bit(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def dimension_based(R,L):
    rd=dimensions(R[0],2)
    ld=dimensions(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=order(R[0]),order(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=iter_bit(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return dictionary_order(R[0],L[0])
       
def generate_shift_table(ori,new):
    count = 0
    idx=[]
    for i in range(len(ori)):
        if ori[new[0][i]] != new[1][i]:
            count += 1
            idx.append(new[1].index(ori[new[0][i]])-i)
        else:
            idx.append(0)
    print('shift count:',count)
    return idx

def generate_run_length_table(idx):
    # [initialization]
    # add end flag
    idx += ['*']  
    # current run
    current = idx[0] 
    # current length
    run_length = 0
    # final output info
    res = []
    # readble output info
    readble = []
    
    # [run_length]
    for i in range(len(idx)):
        if idx[i] == current:
            run_length += 1
        else:
            res += [current,run_length]
            readble.append([current,run_length])
            current = idx[i]
            run_length = 1
    print('run-length table size:',len(res))
    print('run-length table char count:',len(str(res)))
    print('the most large consecutive segment length:',max([*zip(*readble)][1]))
    return res, readble

def test_all(case):
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()  
    print('type: {}'.format(check_type(origin_mapping)))
    
    print()   
    mark,sporadic_seg=generate_idx_list(list(origin_mapping.items()))
    print()
    print('using dimension_based key')
    mmm = list(origin_mapping.items())
    mmm=sorted(mmm,key=cmp_to_key(dimension_based))
    a0,a1 =zip(*mmm)
    aaa = list(zip(a1,[i for i in range(len(mmm))]))
    aaa=sorted(aaa,key=cmp_to_key(dimension_based))
    aaaa = [*zip(*aaa)]
    idx=generate_shift_table(origin_mapping,[list(a0),aaaa[0]])
    res, readble = generate_run_length_table(idx)
    two_level_run_length(res)
    print()
    
if __name__ == '__main__':
    for i in range(0,9,1):
        test_all(i)
    
    document4 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = b"""import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""  
    document1 = """import json as a,sys,re
n,o=sys.argv[1:3]
A=a.load(open(n))
B=[*map(lambda x:[*zip(*x)],[*map(lambda x:sorted(x,key=lambda y:y[1]),[*map(lambda m,n:[*zip(m,n)],A,[*map(lambda x:[*map(lambda y:re.sub(r'(\d+)',lambda z:z.group(1).zfill({}),y),x)],A)])])])]
a.dump(dict(zip(B[0][0],B[1][0])),open(o,'w'))""".format(max(bit_size(origin_mapping.keys()),bit_size(origin_mapping.values())))
    natural_key_method_document='''import json as a,sys,re
n,o=sys.argv[1:3]
a.dump(dict(zip(*[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(n))])),open(o,'w'))'''

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

def two_level_run_length(table):
    flag = False
    temp = []
    tltable = []
    for i in range(0,len(table),2):
        if not flag and table[i+1] == 1:
            temp = [[table[i]],1]
            flag = True
        elif flag and table[i+1] == 1:
            temp[1] += 1
            temp[0] += [table[i]]
        else:
            flag = False
            tltable += temp
            temp = []
            temp += table[i:i+2]
    tltable += temp
    #print(temp)
    print('two level run-length:',len(str(tltable)))

case: 0
bit_size: 3
same: 4452
prefix: 100923
infix: 0
postfix: 0
others: 1
default type
shift count: 0
run-length table size: 2
run-length table char count: 11
the most large consecutive segment length: 105376
two level run-length: 11
pure type
type: 0

731898
consecu idx list char count: 2
sporadic_seg idx list char count: 2

using dimension_based key
shift count: 0
run-length table size: 2
run-length table char count: 11
the most large consecutive segment length: 105376
two level run-length: 11

case: 1
bit_size: 4
same: 448
prefix: 225
infix: 0
postfix: 0
others: 10643
default type
shift count: 10442
run-length table size: 7718
run-length table char count: 28170
the most large consecutive segment length: 429
two level run-length: 21612
using natural key
shift count: 11111
run-length table size: 1830
run-length table char count: 8310
the most large consecutive segment length: 431

two level run-length: 6374
shift count: 11111
type: 2

68102
consecu idx list char count: 50161
sporadi

In [46]:
string3 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]_DR_tob_sai[0][2]_DR_tob_sai[0][3]/unta_$O1/lao1/O$1'
string5 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai_DR_tob_sai_DR_tob_sai_DR_tob_sai/unta_$O1/lao1/O$1'
string4 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]/unta_$O1/lao1/O$1'
string6 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][8]_DR_tob_sai[5][15]/unta_$O1/lao1/O$1'

d='[0-9]+' 
a=lambda x:re.findall(d,x)
c=lambda x:re.sub(r'(\d+)',lambda m:m.group(1).zfill(4),x)
#c=lambda x:''.join(sum(([*i]for i in zip(re.split('(?:\d+)',x),[*map(lambda y:y.zfill(4),re.findall('(\d+)',x))])),[])) if re.findall('(\d+)',x) else x
Q=lambda x,y: 1 if c(x[0])>c(y[0])else -1

fff='''import json as _,sys,re
from functools import cmp_to_key
d='[0-9]+' 
a=lambda x:re.findall(d,x)
c=lambda x:re.sub(d,'',x)+str(len(a(x)))+''.join([*map(lambda y:y.zfill(4),a(x))])
n,o=sys.argv[1:3]
_.dump(dict(zip(*map(lambda t:sorted(list(t),key=cmp_to_key(lambda x,y: 1 if c(x[0])>c(y[0])else -1)),zip(*a.load(open(n)))))),open(o,'w'))'''

print(len(fff))
#mmm = [ it for it in origin_mapping.items() if it[0] not in same ]
# print(len(origin_mapping),len(mmm),len(same))
mmm = list(origin_mapping.items())
mmm=sorted(mmm,key=cmp_to_key(Q))
a0,a1 =zip(*mmm)
aaa = list(zip(a1,[i for i in range(len(mmm))]))

aaa=sorted(aaa,key=cmp_to_key(Q))
generate_run_length_table(generate_shift_table(origin_mapping,[list(a0),[*zip(*aaa)][0]]))



336
run-length table size: 2328
run-length table char count: 11049
the most large consecutive segment length: 353


([8272,
  4,
  8273,
  1,
  8264,
  3,
  8274,
  1,
  8270,
  1,
  8273,
  1,
  8270,
  1,
  8277,
  4,
  8278,
  1,
  8283,
  2,
  8284,
  1,
  8276,
  1,
  8278,
  1,
  8283,
  1,
  8285,
  1,
  8273,
  2,
  8276,
  1,
  8285,
  2,
  8280,
  1,
  8274,
  1,
  8279,
  1,
  8282,
  1,
  8254,
  1,
  8272,
  2,
  8257,
  1,
  8258,
  1,
  8273,
  1,
  8246,
  2,
  8247,
  1,
  8274,
  1,
  8276,
  1,
  8274,
  1,
  8272,
  1,
  8276,
  2,
  8277,
  1,
  8297,
  1,
  8274,
  1,
  8270,
  1,
  8274,
  1,
  8231,
  1,
  8427,
  1,
  8373,
  1,
  8274,
  5,
  8275,
  2,
  8277,
  1,
  8271,
  1,
  8273,
  2,
  8274,
  2,
  8258,
  3,
  8273,
  1,
  8270,
  1,
  8273,
  18,
  8280,
  2,
  8281,
  33,
  8244,
  1,
  8241,
  1,
  8239,
  1,
  8279,
  5,
  8273,
  1,
  8278,
  1,
  8311,
  1,
  8280,
  8,
  8283,
  1,
  8133,
  1,
  8222,
  1,
  8129,
  1,
  8280,
  1,
  8276,
  1,
  8214,
  1,
  8263,
  1,
  8277,
  17,
  8196,
  1,
  8310,
  1,
  8242,
  1,
  8275,
  32,
  8277,
  8,
  8279,
